### TLOF Layout Generator with Selective JSON Output with Memory by GROK 

In [ ]:
import os
import json
from datetime import datetime
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
    #api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# History file path
HISTORY_FILE = "tlof_history.json"

# Function to load history
def load_history():
    if os.path.exists(HISTORY_FILE):
        with open(HISTORY_FILE, 'r') as f:
            return json.load(f)
    return []

# Function to save history
def save_history(entry):
    history = load_history()
    history.append(entry)
    with open(HISTORY_FILE, 'w') as f:
        json.dump(history, f, indent=2)

# Function to get the last configuration
def get_last_config():
    history = load_history()
    if history:
        return history[-1]['output']
    return None

# Define function schema
function_def = [
    {
        "name": "generate_landing_surface_layout",
        "description": "Generate a structured JSON layout for a TLOF (Touchdown and Lift-Off Area)...",
        "parameters": {
            "type": "object",
            "properties": {
                "aircraft": {"type": "string", "default": None},
                "diameter": {"type": "number", "default": 30.0, "minimum": 1.0, "maximum": 200.0},
                "shape_type": {"type": "string", "enum": ["Circle", "Polygon", "Rectangle"], "default": "Rectangle"},
                "position": {"type": "array", "items": {"type": "number"}, "minItems": 2, "maxItems": 2},
                "sides": {"type": "number", "default": 4, "minimum": 3, "maximum": 70},
                "width": {"type": "number", "default": 30, "minimum": 30, "maximum": 30},
                "length": {"type": "number", "default": 30, "minimum": 30, "maximum": 30},
                "height": {"type": "number", "default": 0.01, "minimum": 0.01, "maximum": 5},
                "rotation": {"type": "number", "default": 0, "minimum": 0, "maximum": 359},
                "transparency": {"type": "number", "default": 1, "minimum": 0, "maximum": 1},
                "elevation": {"type": "number", "default": 0, "minimum": 0, "maximum": 1000},
                "markingType": {"type": "string", "enum": ["solid", "dashed"], "default": "solid"},
                "markingColor": {"type": "string", "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"], "default": "white"},
                "markingThickness": {"type": "number", "default": 0.5, "minimum": 0.1, "maximum": 1.5},
                "dashLength": {"type": "number", "default": 1, "minimum": 0.5, "maximum": 5},
                "dashDistance": {"type": "number", "default": 1, "minimum": 0.5, "maximum": 5},
                "landingMarker": {"type": "string", "default": None},
                "markerScale": {"type": "number", "default": 1, "minimum": 0.5, "maximum": 50},
                "markerThickness": {"type": "number", "default": 0.02, "minimum": 0, "maximum": 1},
                "markerRotation": {"type": "number", "default": 0, "minimum": 0, "maximum": 359},
                "markerColor": {"type": "string", "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"], "default": "white"},
                "letterThickness": {"type": "number", "default": 0.5, "minimum": 0, "maximum": 50},
                "tdpcCategory": {"type": "boolean", "default": False},
                "tdpcType": {"type": "string", "enum": ["circle", "cross", "square"], "default": "circle"},
                "tdpcScale": {"type": "number", "default": 5, "minimum": 0.5, "maximum": 50},
                "tdpcThickness": {"type": "number", "default": 0.5, "minimum": 0.1, "maximum": 50},
                "tdpcExtrusion": {"type": "number", "default": 0.02, "minimum": 0, "maximum": 1},
                "tdpcRotation": {"type": "number", "default": 0, "minimum": 0, "maximum": 359},
                "tdpcColor": {"type": "string", "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"], "default": "white"},
                "lightColor": {"type": "string", "default": None},
                "lightScale": {"type": "number", "default": 1, "minimum": -20, "maximum": 100},
                "lightDistance": {"type": "number", "default": 1, "minimum": 1, "maximum": 50},
                "lightRadius": {"type": "number", "default": 0.3, "minimum": 0.1, "maximum": 1},
                "lightHeight": {"type": "number", "default": 0.2, "minimum": 0.1, "maximum": 0.25},
                "safetyAreaType": {"type": "string", "enum": ["offset", "multiplier"], "default": "offset"},
                "dValue": {"type": "number", "default": 10, "minimum": 1, "maximum": 20},
                "multiplier": {"type": "number", "default": 1.5, "minimum": 0.5, "maximum": 20},
                "offsetDistance": {"type": "number", "default": 3, "minimum": 0, "maximum": 50},
                "curveAngle": {"type": "number", "default": 45, "minimum": 0, "maximum": 90},
                "netHeight": {"type": "number", "default": 15, "minimum": 0, "maximum": 100},
                "safetyNetTransparency": {"type": "number", "default": 0.5, "minimum": 0, "maximum": 1}
            },
            "required": []
        }
    }
]

# Function to enforce schema constraints
def enforce_schema_constraints(args, schema):
    properties = schema[0]["parameters"]["properties"]
    constrained_args = {}
    for key, value in args.items():
        prop = properties.get(key, {})
        if isinstance(value, (int, float)) and "minimum" in prop and "maximum" in prop:
            constrained_args[key] = max(prop["minimum"], min(value, prop["maximum"]))
        else:
            constrained_args[key] = value
    return constrained_args

# Function to get OpenAI response
def get_function_call(prompt: str):
    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
        messages=[{"role": "user", "content": prompt}],
        functions=function_def,
        function_call={"name": "generate_landing_surface_layout"}
    )
    args = json.loads(response.choices[0].message.function_call.arguments)
    return enforce_schema_constraints(args, function_def)

# Function to merge update with previous configuration
def merge_with_previous(new_args, prev_config, prompt):
    if prev_config is None or "update" not in prompt.lower():
        return new_args
    merged_args = prev_config.copy()
    merged_args.update(new_args)
    return enforce_schema_constraints(merged_args, function_def)

# Main loop
while True:
    try:
        user_input = input("Describe the TLOF (Landing Surface) in natural language:\n")
        
        # Step 1: Get user-intended parameters
        user_args = get_function_call(user_input)

        # Step 2: Merge with previous configuration if it's an update
        last_config = get_last_config()
        user_args = merge_with_previous(user_args, last_config, user_input)

        # Step 3: Save to history (store full configuration)
        history_entry = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "input": user_input,
            "output": user_args
        }
        save_history(history_entry)

        # Step 4: Print full configuration
        print(json.dumps(user_args, indent=2))

    except KeyboardInterrupt:
        print("\nExiting program.")
        break
    except Exception as e:
        print(f"Error: {e}")
        continue

Describe the TLOF (Landing Surface) in natural language:
 Generate a rectangular TLOF for a tiltrotor aircraft with 30m x 40m dimensions, elevation 5m, rotation 15 degrees, and 0.6 transparency. Location is [139.6917, 35.6895]. Add a 'V' landing marker in blue, scaled to 8, rotated to 90 degrees


{
  "aircraft": "tiltrotor",
  "shape_type": "Rectangle",
  "position": [
    139.6917,
    35.6895
  ],
  "width": 30,
  "length": 30,
  "height": 0.01,
  "rotation": 15,
  "transparency": 0.6,
  "elevation": 5,
  "landingMarker": "V",
  "markerScale": 8,
  "markerThickness": 0.02,
  "markerRotation": 90,
  "markerColor": "blue"
}


Describe the TLOF (Landing Surface) in natural language:
 update rotation to 30


{
  "aircraft": "tiltrotor",
  "shape_type": "Rectangle",
  "position": [
    139.6917,
    35.6895
  ],
  "width": 30,
  "length": 30,
  "height": 0.01,
  "rotation": 30,
  "transparency": 0.6,
  "elevation": 5,
  "landingMarker": "V",
  "markerScale": 8,
  "markerThickness": 0.02,
  "markerRotation": 90,
  "markerColor": "blue"
}


Describe the TLOF (Landing Surface) in natural language:
 set light height to 0.21


{
  "lightHeight": 0.21
}


Describe the TLOF (Landing Surface) in natural language:
 update widht to 19


{
  "lightHeight": 0.21,
  "width": 30
}


Describe the TLOF (Landing Surface) in natural language:
 set length to 29


{
  "length": 30
}

Exiting program.


### TLOF Layout Generator with Selective JSON Output by GROK 

In [31]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Prompt user input
user_input = input("Describe the TLOF (Landing Surface) in natural language:\n")

# Define function schema
function_def = [
    {
        "name": "generate_landing_surface_layout",
        "description": "Generate a structured JSON layout for a TLOF (Touchdown and Lift-Off Area)...",
        "parameters": {
            "type": "object",
            "properties": {
                "aircraft": {"type": "string", "default": None},
                "diameter": {"type": "number", "default": 30.0, "minimum": 1.0, "maximum": 200.0},
                "shape_type": {"type": "string", "enum": ["Circle", "Polygon", "Rectangle"], "default": "Rectangle"},
                "position": {"type": "array", "items": {"type": "number"}, "minItems": 2, "maxItems": 2},
                "sides": {"type": "number", "default": 4, "minimum": 3, "maximum": 70},
                "width": {"type": "number", "default": 30, "minimum": 30, "maximum": 30},
                "length": {"type": "number", "default": 30, "minimum": 30, "maximum": 30},
                "height": {"type": "number", "default": 0.01, "minimum": 0.01, "maximum": 5},
                "rotation": {"type": "number", "default": 0, "minimum": 0, "maximum": 359},
                "transparency": {"type": "number", "default": 1, "minimum": 0, "maximum": 1},
                "elevation": {"type": "number", "default": 0, "minimum": 0, "maximum": 1000},
                "markingType": {"type": "string", "enum": ["solid", "dashed"], "default": "solid"},
                "markingColor": {"type": "string", "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"], "default": "white"},
                "markingThickness": {"type": "number", "default": 0.5, "minimum": 0.1, "maximum": 1.5},
                "dashLength": {"type": "number", "default": 1, "minimum": 0.5, "maximum": 5},
                "dashDistance": {"type": "number", "default": 1, "minimum": 0.5, "maximum": 5},
                "landingMarker": {"type": "string", "default": None},
                "markerScale": {"type": "number", "default": 1, "minimum": 0.5, "maximum": 50},
                "markerThickness": {"type": "number", "default": 0.02, "minimum": 0, "maximum": 1},
                "markerRotation": {"type": "number", "default": 0, "minimum": 0, "maximum": 359},
                "markerColor": {"type": "string", "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"], "default": "white"},
                "letterThickness": {"type": "number", "default": 0.5, "minimum": 0, "maximum": 50},
                "tdpcCategory": {"type": "boolean", "default": False},
                "tdpcType": {"type": "string", "enum": ["circle", "cross", "square"], "default": "circle"},
                "tdpcScale": {"type": "number", "default": 5, "minimum": 0.5, "maximum": 50},
                "tdpcThickness": {"type": "number", "default": 0.5, "minimum": 0.1, "maximum": 50},
                "tdpcExtrusion": {"type": "number", "default": 0.02, "minimum": 0, "maximum": 1},
                "tdpcRotation": {"type": "number", "default": 0, "minimum": 0, "maximum": 359},
                "tdpcColor": {"type": "string", "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"], "default": "white"},
                "lightColor": {"type": "string", "default": None},
                "lightScale": {"type": "number", "default": 1, "minimum": -20, "maximum": 100},
                "lightDistance": {"type": "number", "default": 1, "minimum": 1, "maximum": 50},
                "lightRadius": {"type": "number", "default": 0.3, "minimum": 0.1, "maximum": 1},
                "lightHeight": {"type": "number", "default": 0.2, "minimum": 0.1, "maximum": 0.25},
                "safetyAreaType": {"type": "string", "enum": ["offset", "multiplier"], "default": "offset"},
                "dValue": {"type": "number", "default": 10, "minimum": 1, "maximum": 20},
                "multiplier": {"type": "number", "default": 1.5, "minimum": 0.5, "maximum": 20},
                "offsetDistance": {"type": "number", "default": 3, "minimum": 0, "maximum": 50},
                "curveAngle": {"type": "number", "default": 45, "minimum": 0, "maximum": 90},
                "netHeight": {"type": "number", "default": 15, "minimum": 0, "maximum": 100},
                "safetyNetTransparency": {"type": "number", "default": 0.5, "minimum": 0, "maximum": 1}
            },
            "required": []
        }
    }
]

# Function to enforce schema constraints
def enforce_schema_constraints(args, schema):
    properties = schema[0]["parameters"]["properties"]
    constrained_args = {}
    for key, value in args.items():
        prop = properties.get(key, {})
        if isinstance(value, (int, float)) and "minimum" in prop and "maximum" in prop:
            constrained_args[key] = max(prop["minimum"], min(value, prop["maximum"]))
        else:
            constrained_args[key] = value
    return constrained_args

# Function to get OpenAI response
def get_function_call(prompt: str):
    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
        messages=[{"role": "user", "content": prompt}],
        functions=function_def,
        function_call={"name": "generate_landing_surface_layout"}
    )
    args = json.loads(response.choices[0].message.function_call.arguments)
    return enforce_schema_constraints(args, function_def)

# Step 1: Get user-intended parameters
user_args = get_function_call(user_input)

# Step 2: Get default values for comparison
default_args = get_function_call("Generate default TLOF")

# Step 3: Compare user_args with default_args
def get_modified_fields(user_args, default_args):
    modified = {}
    for key, user_value in user_args.items():
        default_value = default_args.get(key)
        if user_value != default_value:
            modified[key] = user_value
    return modified

filtered_output = get_modified_fields(user_args, default_args)

# Step 4: Print result
if filtered_output:
    print(json.dumps(filtered_output, indent=2))
else:
    print("⚠️ No fields were modified compared to the default output.")

Describe the TLOF (Landing Surface) in natural language:
 Generate a rectangular TLOF for a tiltrotor aircraft with 30m x 40m dimensions, elevation 5m, rotation 15 degrees, and 0.6 transparency. Location is [139.6917, 35.6895]. Add a 'V' landing marker in blue, scaled to 8, rotated to 90 degrees


{
  "aircraft": "tiltrotor",
  "width": 30,
  "length": 30,
  "elevation": 5,
  "rotation": 15,
  "transparency": 0.6,
  "position": [
    139.6917,
    35.6895
  ],
  "shape_type": "Rectangle",
  "landingMarker": "V",
  "markerScale": 8,
  "markerRotation": 90,
  "markerColor": "blue"
}


### TLOF Layout Generator with Selective JSON Output by ChatGPT

In [23]:
import os
import time
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Prompt user input
user_input = input("Describe the TLOF (Landing Surface) in natural language:\n")

# Define function schema
function_def = [
    {
        "name": "generate_landing_surface_layout",
        "description": "Generate a structured JSON layout for a TLOF (Touchdown and Lift-Off Area)...",
        "parameters": {
            "type": "object",
            "properties": {
                "aircraft": {"type": "string"},
                "diameter": {"type": "number"},
                "shape_type": {"type": "string", "enum": ["Circle", "Polygon", "Rectangle"]},
                "position": {"type": "array", "items": {"type": "number"}, "minItems": 2, "maxItems": 2},
                "sides": {"type": "number"},
                "surface_width": {"type": "number"},
                "surface_length": {"type": "number"},
                "surface_height": {"type": "number"},
                "rotation": {"type": "number"},
                "transparency": {"type": "number"},
                "elevation": {"type": "number"},
                "markingType": {"type": "string", "enum": ["solid", "dashed"]},
                "markingColor": {"type": "string", "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"]},
                "markingThickness": {"type": "number"},
                "dashLength": {"type": "number"},
                "dashDistance": {"type": "number"},
                "landingMarker": {"type": "string"},
                "markerScale": {"type": "number"},
                "markerThickness": {"type": "number"},
                "markerRotation": {"type": "number"},
                "markerColor": {"type": "string", "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"]},
                "letterThickness": {"type": "number"},
                "tdpcCategory": {"type": "boolean"},
                "tdpcType": {"type": "string", "enum": ["circle", "cross", "square"]},
                "tdpcScale": {"type": "number"},
                "tdpcThickness": {"type": "number"},
                "tdpcExtrusion": {"type": "number"},
                "tdpcRotation": {"type": "number"},
                "tdpcColor": {"type": "string", "enum": ["white", "yellow", "blue", "red", "green", "black", "purple", "orange", "gray", "brown"]},
                "lightColor": {"type": "string"},
                "lightScale": {"type": "number"},
                "lightDistance": {"type": "number"},
                "lightRadius": {"type": "number"},
                "lightHeight": {"type": "number"},
                "safetyAreaType": {"type": "string", "enum": ["offset", "multiplier"]},
                "dValue": {"type": "number"},
                "multiplier": {"type": "number"},
                "offsetDistance": {"type": "number"},
                "curveAngle": {"type": "number"},
                "netHeight": {"type": "number"},
                "safetyNetTransparency": {"type": "number"}
            },
            "required": [
                #"aircraft", "surface_length", "surface_width", "surface_height",
                #"sides", "diameter", "markingThickness", "rotation", "transparency",
                #"shape_type", "position"
            ]
        }
    }
]

# Function to get OpenAI response
def get_function_call(prompt: str):
    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_MODEL_NAME"),
        messages=[{"role": "user", "content": prompt}],
        functions=function_def,
        function_call={"name": "generate_landing_surface_layout"}
    )
    return json.loads(response.choices[0].message.function_call.arguments)

# Step 1: Get user-intended parameters
user_args = get_function_call(user_input)

# Step 2: Get default values for comparison
default_args = get_function_call("Generate default TLOF")

# Step 3: Compare user_args with default_args
def get_modified_fields(user_args, default_args):
    modified = {}
    for key, user_value in user_args.items():
        default_value = default_args.get(key)
        if user_value != default_value:
            modified[key] = user_value
    return modified

filtered_output = get_modified_fields(user_args, default_args)

# Step 4: Print result
if filtered_output:
    print(json.dumps(filtered_output, indent=2))
else:
    print("⚠️ No fields were modified compared to the default output.")


Describe the TLOF (Landing Surface) in natural language:
 set height to 3


{
  "surface_height": 3
}
